# Comparativa de Word2Vec, GloVe y FastText para obtener la similaridad semántica entre pares de textos
## Pablo Valdunciel Sánchez 


## 1. Modelos

Utilizamos el modelo *KeyedVectors* de la librería [*gesim*](https://radimrehurek.com/gensim/index.html) para cargar los vectores pre-entrenados de los diferentes modelos 

In [1]:
from gensim.models import KeyedVectors 

Cargamos los vectores pre-entrenados con modelos Word2Vec, GloVe y FastText. Los vectores pre-entrenados de cada modelo utilizados son:

- **Word2Vec**:  [GoogleNews-vectors-negative300.bin.gz](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)  
- **GloVe**: [Common  Crawl  (840B  tokens,2.2M vocab, cased, 300d vectors)](http://nlp.stanford.edu/data/glove.840B.300d.zip)
- **FastText**: [rawl-300d-2M.vec.zip:  2  million  word  vectors  trained  on  Common  Crawl  (600Btokens)](https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip)

En el caso de los vectores de GloVe, se ha utilizado la función [*gensim.scripts.glove2word2vec.glove2word2vec*](https://radimrehurek.com/gensim/scripts/glove2word2vec.html) para convertir el archivo al formato Word2Vec.


In [2]:
PATH_WORD2VEC = './data/embedding/word2vec/GoogleNews-vectors-negative300.bin'
PATH_GLOVE = './data/embedding/glove/glove.840B.300d.w2v.txt'
PATH_FASTTEXT = './data/embedding/fasttext/crawl-300d-2M.vec'

Cargar los vectores puede llevar varios minutos.

In [ ]:
word2vec = KeyedVectors.load_word2vec_format(PATH_WORD2VEC, binary=True)

In [36]:
glove = KeyedVectors.load_word2vec_format(PATH_GLOVE, binary=False)

In [3]:
fasttext = KeyedVectors.load_word2vec_format(PATH_FASTTEXT, binary=False)

## 2. Datos

Haciendo uso de las funciones del módulo *load.py* cargamos los conjuntos de test de las tareas STS12, STS13, STS14, STS15 y STS16. Estas funciones llevan a cabo un preprocesamiento de las oraciones según los parámetros que se indiquen. Entre las posibilidades de preprocesamiento se encuentran:

- **lowercase**: hacer que todas las palabras estén en minúscula.
- **stop_words**: eliminar las palabras que no aportan casi significado semántico como determinantes, preprosiciones, etc.
- **punctuation**: elminar los símbolos de puntuación.
- **only_ascci**: eliminar las palabras que no estén formadas por caracteres ASCII.
- **lemmatization**: sustituir las palabras por su lexema.

El preprocesamiento del texto está implementado en la función *preprocess* en el módulo *utils.py*. La función *preprocess* hace uso de la librería [spaCy](https://spacy.io/) para llevar a cabo el preprocesamiento.

In [4]:
from load import load_sts_12, load_sts_13, load_sts_14, load_sts_15, load_sts_16
from load import load_frequencies

En este caso no se aplica ningún tipo de preprocesamiento.

In [52]:
PATH_DATASETS = './data/datasets/STS'
PATH_FREQUENCIES = './data/frequencies.tsv'
PREPROCESSING =  {'lowercase':  False, 
                  'stop_words': False, 
                  'punctuation': False, 
                  'only_ascii': False, 
                  'lemmatization': False
                 }

Cargamos también las frecuencias de las palabras en el corpus par poder aplicar el SIF.

In [53]:
freqs = load_frequencies(PATH_FREQUENCIES)

In [54]:
sts12 = load_sts_12(PATH_DATASETS, PREPROCESSING)
sts13 = load_sts_13(PATH_DATASETS, PREPROCESSING)
sts14 = load_sts_14(PATH_DATASETS, PREPROCESSING)
sts15 = load_sts_15(PATH_DATASETS, PREPROCESSING)
sts16 = load_sts_16(PATH_DATASETS, PREPROCESSING)


***** TASK: STS12 *****

Preprocessing -MSRpar-
-MSRpar- preprocessed correctly
Preprocessing -MSRvid-
-MSRvid- preprocessed correctly
Preprocessing -SMTeuroparl-
-SMTeuroparl- preprocessed correctly
Preprocessing -surprise.OnWN-
-surprise.OnWN- preprocessed correctly
Preprocessing -surprise.SMTnews-
-surprise.SMTnews- preprocessed correctly

***** TASK: STS13 (-SMT) ***


Preprocessing -FNWN-
-FNWN- preprocessed correctly
Preprocessing -headlines-
-headlines- preprocessed correctly
Preprocessing -OnWN-
-OnWN- preprocessed correctly

***** TASK: STS14 *****

Preprocessing -deft-forum-
-deft-forum- preprocessed correctly
Preprocessing -deft-news-
-deft-news- preprocessed correctly
Preprocessing -headlines-
-headlines- preprocessed correctly
Preprocessing -images-
-images- preprocessed correctly
Preprocessing -OnWN-
-OnWN- preprocessed correctly
Preprocessing -tweet-news-
-tweet-news- preprocessed correctly

***** TASK: STS15 *****

Preprocessing -answers-forums-
-answers-forums- prepro

## 3. Métodos

In [55]:
from functools import partial
from methods import avg_cosine, wmd, sif_cosine

In [56]:
METHODS = [
    ("Word2Vec + AVG", partial(avg_cosine, model=word2vec)),
    ("Word2Vec + WMD", partial(wmd, model=word2vec)),
    ("Word2Vec + SIF", partial(sif_cosine, model=word2vec, frequencies=freqs, a=0.001))
    #("GloVe + AVG", partial(avg_cosine, model=glove)),
    #("GloVe + WMD", partial(wmd, model=glove)),
    #("GloVe + SIF", partial(sif_cosine, model=glove, frequencies=freqs, a=0.001)),
    #("FastText + AVG", partial(avg_cosine, model=fasttext)),
    #("FastText + WMD", partial(wmd, model=fasttext)),
    #("FastText + SIF", partial(sif_cosine, model=fasttext, frequencies=freqs, a=0.001))
]

## 4. Evaluación

In [57]:
from utils import evaluate

In [58]:
word2vec_sts12_pearson, sts12_spearman = evaluate(sts12, METHODS)
word2vec_sts13_pearson, sts13_spearman = evaluate(sts13, METHODS)
word2vec_sts14_pearson, sts14_spearman = evaluate(sts14, METHODS)
word2vec_sts15_pearson, sts15_spearman = evaluate(sts15, METHODS)
word2vec_sts16_pearson, sts16_spearman = evaluate(sts16, METHODS)

In [64]:
fasttext_sts16_pearson

{'FastText + AVG': 0.6326237499753964,
 'FastText + WMD': 0.6312212885648231,
 'FastText + SIF': 0.7315395720676849}

In [44]:
print("++++ Task STS12 ++++")
sts12_pearson

++++ Task STS12 ++++


{'FastText + AVG': 0.6198524200427709,
 'FastText + WMD': 0.5280347733615505,
 'FastText + SIF': 0.6219464875863289}

In [45]:
print("++++ Task STS13 ++++")
sts13_pearson

++++ Task STS13 ++++


{'FastText + AVG': 0.690635771639227,
 'FastText + WMD': 0.4012119583863098,
 'FastText + SIF': 0.7557200532758551}

In [46]:
print("++++ Task STS14 ++++")
sts14_pearson

++++ Task STS14 ++++


{'FastText + AVG': 0.7264079006748914,
 'FastText + WMD': 0.567738188273139,
 'FastText + SIF': 0.7492322179215466}

In [47]:
print("++++ Task STS15 ++++")
sts15_pearson

++++ Task STS15 ++++


{'FastText + AVG': 0.7508655344369192,
 'FastText + WMD': 0.6738544035294207,
 'FastText + SIF': 0.7609634784830468}

In [48]:
print("++++ Task STS16 ++++")
sts15_pearson

++++ Task STS16 ++++


{'FastText + AVG': 0.7508655344369192,
 'FastText + WMD': 0.6738544035294207,
 'FastText + SIF': 0.7609634784830468}